In [ ]:
import numpy as np
import pyci

In [ ]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.AOint('6-31g*', 'licn.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('mo_scf_info.npz')['eps_a']
Ca = np.load('mo_scf_info.npz')['Ca'] 
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints)
del ao_erints

In [ ]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

In [ ]:
orbinfo = (16, nmo)
active_space = (2,2)
singles = True
full_cis = True
doubles = True
options = singles, full_cis, doubles
scf_energy = aoint.scf_energy
HCISD = pyci.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=8)